- title: Using Jupyter Notebook with Pelican in Windows
- author: Jianghao Chu
- date: 2020-11-26
- category: python
- tags: Jupyter-Notebook, Pelican

Update: This solution does not work with nbconvert >= 6. Tested on nbconvert == 5.6.1.

[Jupyter-Pelican](https://github.com/danielfrg/pelican-jupyter) is a plugin that helps Pelican to generate static contents from Jupyter notebook. I like it because it allows the user to put the
metadata in the first cell of the notebook. No need to create a separate file for the metadata or edit the metadata of the notebook. However, this option will encounter a permission denied error on Windows OS because Windows does not allow (at least easily) openning a temporary file for a second time, according to the documentation of [tempfile.NamedTemporaryFile](https://docs.python.org/3/library/tempfile.html#tempfile.NamedTemporaryFile). Other discussions on reading a temporary file in Windows can be found at [Python bug tracker](https://bugs.python.org/issue14243#msg164504) and [stackoverflow](https://stackoverflow.com/questions/15169101/how-to-create-a-temporary-file-that-can-be-read-by-a-subprocess).

The [lines](https://github.com/danielfrg/pelican-jupyter/blob/183d01824788dcc5fe9320f746f9c8c62aa29999/pelican_jupyter/markup.py#L78-L82) causing the error are as below:
```python
with tempfile.NamedTemporaryFile("w+", encoding="utf-8") as metadata_file:
    md_reader = MarkdownReader(self.settings)
    metadata_file.write(metacell)
    metadata_file.flush()
    _content, metadata = md_reader.read(metadata_file.name)
```
Right now, there is no perfect fix. So I went ahead and changed the lines in my local library as below:
```python
metadata_file = tempfile.NamedTemporaryFile("w+", encoding="utf-8", delete=False)
try:
    md_reader = MarkdownReader(self.settings)
    metadata_file.write(metacell)
    metadata_file.flush()
    metadata_file.close()
    _content, metadata = md_reader.read(metadata_file.name)
finally:
    os.remove(metadata_file.name)
```
It's not an elegant solution but now everything works. I can have Jupyter notebook organize all my Markdown, LaTex, input and output while having Pelican generate the static site. Below are a few tests to see it works.

#### Markdown
I like Markdown.
#### LaTex
This is a try to use LaTeX in Markdown. Let us start with a simple linear model.
$$y=\beta_0+\beta_1x_1+\beta_2x_2+u$$

In [1]:
print('Hello World!')

Hello World!


In [2]:
import numpy as np
a = np.array([1, 2, 3])   # Create a rank 1 array
print(type(a))            # Prints "<class 'numpy.ndarray'>"
print(a.shape)            # Prints "(3,)"
print(a[0], a[1], a[2])   # Prints "1 2 3"
a[0] = 5                  # Change an element of the array
print(a)                  # Prints "[5, 2, 3]"

b = np.array([[1,2,3],[4,5,6]])    # Create a rank 2 array
print(b.shape)                     # Prints "(2, 3)"
print(b[0, 0], b[0, 1], b[1, 0])   # Prints "1 2 4"

<class 'numpy.ndarray'>
(3,)
1 2 3
[5 2 3]
(2, 3)
1 2 4
